In [2]:
import sys
import sktime
import tqdm as tq
import xgboost as xgb
import matplotlib
import seaborn as sns
import sklearn as skl
import pandas as pd
import numpy as np

In [3]:
import matplotlib.pyplot as plt
from tqdm import tqdm
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.utils.plotting import plot_series
from xgboost import XGBRegressor

pd.set_option('display.max_columns', 30)

In [4]:
train = pd.read_csv('./data/train.csv')
train.head()

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76
4,1_20220601 04,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40


In [5]:
test = pd.read_csv('./data/test.csv')
test.head()

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%)
0,1_20220825 00,1,20220825 00,23.5,0.0,2.2,72
1,1_20220825 01,1,20220825 01,23.0,0.0,0.9,72
2,1_20220825 02,1,20220825 02,22.7,0.0,1.5,75
3,1_20220825 03,1,20220825 03,22.1,0.0,1.3,78
4,1_20220825 04,1,20220825 04,21.8,0.0,1.0,77


In [6]:
## 변수 영문명으로 변경
cols = ['num_date_time','num', 'date_time', 'temp', 'prec','wind' ,'hum', 'hr', 'mj/m2', 'power']
train.columns = cols
train.head()

,num_date_time,num,date_time,temp,prec,wind,hum,hr,mj/m2,power
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76
4,1_20220601 04,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40


### 결측치 처리


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204000 entries, 0 to 203999
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   num_date_time  204000 non-null  object 
 1   num            204000 non-null  int64  
 2   date_time      204000 non-null  object 
 3   temp           204000 non-null  float64
 4   prec           43931 non-null   float64
 5   wind           203981 non-null  float64
 6   hum            203991 non-null  float64
 7   hr             128818 non-null  float64
 8   mj/m2          116087 non-null  float64
 9   power          204000 non-null  float64
dtypes: float64(7), int64(1), object(2)
memory usage: 15.6+ MB


In [8]:
train.isnull().sum()

num_date_time         0
num                   0
date_time             0
temp                  0
prec             160069
wind                 19
hum                   9
hr                75182
mj/m2             87913
power                 0
dtype: int64

In [9]:
# 강수량 결측치 0.0으로 채우기
train['prec'].fillna(0.0, inplace=True)

In [10]:
# pandas 내 선형보간 method 사용 wind, hum 열에 적용
for i in range(100):
    train.iloc[i*2040:(i+1)*2040, 5:7]  = train.iloc[i*2040:(i+1)*2040, 5:7].interpolate()

In [12]:
train.head()

,num_date_time,num,date_time,temp,prec,wind,hum,hr,mj/m2,power
0,1_20220601 00,1,20220601 00,18.6,0.0,0.9,42.0,NaN,NaN,1085.28
1,1_20220601 01,1,20220601 01,18.0,0.0,1.1,45.0,NaN,NaN,1047.36
2,1_20220601 02,1,20220601 02,17.7,0.0,1.5,45.0,NaN,NaN,974.88
3,1_20220601 03,1,20220601 03,16.7,0.0,1.4,48.0,NaN,NaN,953.76
4,1_20220601 04,1,20220601 04,18.4,0.0,2.8,43.0,NaN,NaN,986.40


In [11]:
train.isnull().sum()

num_date_time        0
num                  0
date_time            0
temp                 0
prec                 0
wind                 0
hum                  0
hr               75182
mj/m2            87913
power                0
dtype: int64

In [13]:
# train.csv 에만 있는 열 제거 num_date_time, hr, mj/m2
train = train.drop(['num_date_time','hr','mj/m2'], axis=1)
train.head()

,num,date_time,temp,prec,wind,hum,power
0,1,20220601 00,18.6,0.0,0.9,42.0,1085.28
1,1,20220601 01,18.0,0.0,1.1,45.0,1047.36
2,1,20220601 02,17.7,0.0,1.5,45.0,974.88
3,1,20220601 03,16.7,0.0,1.4,48.0,953.76
4,1,20220601 04,18.4,0.0,2.8,43.0,986.40


In [14]:
# 시간 관련 변수들 생성
date = pd.to_datetime(train.date_time)
train['hour'] = date.dt.hour
train['day'] = date.dt.weekday
train['month'] = date.dt.month
train['week'] = date.dt.weekofyear
train.head()

C:\Users\User\AppData\Local\Temp\ipykernel_39792\980611340.py:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  train['week'] = date.dt.weekofyear


,num,date_time,temp,prec,wind,hum,power,hour,day,month,week
0,1,20220601 00,18.6,0.0,0.9,42.0,1085.28,0,2,6,22
1,1,20220601 01,18.0,0.0,1.1,45.0,1047.36,1,2,6,22
2,1,20220601 02,17.7,0.0,1.5,45.0,974.88,2,2,6,22
3,1,20220601 03,16.7,0.0,1.4,48.0,953.76,3,2,6,22
4,1,20220601 04,18.4,0.0,2.8,43.0,986.40,4,2,6,22


In [15]:
## 건물별, 요일별, 시간별 발전량 평균 넣어주기
power_mean = pd.pivot_table(train, values = 'power', index = ['num', 'hour', 'day'], aggfunc = np.mean).reset_index()
tqdm.pandas()
train['day_hour_mean'] = train.progress_apply(lambda x : power_mean.loc[(power_mean.num == x['num']) & (power_mean.hour == x['hour']) & (power_mean.day == x['day']) ,'power'].values[0], axis = 1)
train.head()

100%|██████████| 204000/204000 [03:07<00:00, 1089.93it/s]


,num,date_time,temp,prec,wind,hum,power,hour,day,month,week,day_hour_mean
0,1,20220601 00,18.6,0.0,0.9,42.0,1085.28,0,2,6,22,1774.744615
1,1,20220601 01,18.0,0.0,1.1,45.0,1047.36,1,2,6,22,1687.347692
2,1,20220601 02,17.7,0.0,1.5,45.0,974.88,2,2,6,22,1571.483077
3,1,20220601 03,16.7,0.0,1.4,48.0,953.76,3,2,6,22,1522.153846
4,1,20220601 04,18.4,0.0,2.8,43.0,986.40,4,2,6,22,1506.793846


In [16]:
## 건물별 시간별 발전량 평균 넣어주기
power_hour_mean = pd.pivot_table(train, values = 'power', index = ['num', 'hour'], aggfunc = np.mean).reset_index()
tqdm.pandas()
train['hour_mean'] = train.progress_apply(lambda x : power_hour_mean.loc[(power_hour_mean.num == x['num']) & (power_hour_mean.hour == x['hour']) ,'power'].values[0], axis = 1)
train.head()

100%|██████████| 204000/204000 [02:16<00:00, 1499.52it/s]


,num,date_time,temp,prec,wind,hum,power,hour,day,month,week,day_hour_mean,hour_mean
0,1,20220601 00,18.6,0.0,0.9,42.0,1085.28,0,2,6,22,1774.744615,1706.318118
1,1,20220601 01,18.0,0.0,1.1,45.0,1047.36,1,2,6,22,1687.347692,1622.620235
2,1,20220601 02,17.7,0.0,1.5,45.0,974.88,2,2,6,22,1571.483077,1506.971294
3,1,20220601 03,16.7,0.0,1.4,48.0,953.76,3,2,6,22,1522.153846,1437.365647
4,1,20220601 04,18.4,0.0,2.8,43.0,986.40,4,2,6,22,1506.793846,1447.321412


In [17]:
## 건물별 시간별 발전량 표준편차 넣어주기
power_hour_std = pd.pivot_table(train, values = 'power', index = ['num', 'hour'], aggfunc = np.std).reset_index()
tqdm.pandas()
train['hour_std'] = train.progress_apply(lambda x : power_hour_std.loc[(power_hour_std.num == x['num']) & (power_hour_std.hour == x['hour']) ,'power'].values[0], axis = 1)
train.head()

100%|██████████| 204000/204000 [01:57<00:00, 1740.32it/s]


,num,date_time,temp,prec,wind,hum,power,hour,day,month,week,day_hour_mean,hour_mean,hour_std
0,1,20220601 00,18.6,0.0,0.9,42.0,1085.28,0,2,6,22,1774.744615,1706.318118,446.882767
1,1,20220601 01,18.0,0.0,1.1,45.0,1047.36,1,2,6,22,1687.347692,1622.620235,439.662704
2,1,20220601 02,17.7,0.0,1.5,45.0,974.88,2,2,6,22,1571.483077,1506.971294,412.071906
3,1,20220601 03,16.7,0.0,1.4,48.0,953.76,3,2,6,22,1522.153846,1437.365647,391.205981
4,1,20220601 04,18.4,0.0,2.8,43.0,986.40,4,2,6,22,1506.793846,1447.321412,381.099697


### 공휴일 변수 추가, 대체공휴일 적용

In [18]:
### 공휴일 변수 추가
### 공휴일이면 1 아니면 0
### 6월 1일, 6월 6일, 8월 15일 임시공휴일로 지정이므로 1로 변경
train['holiday'] = train.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)
train.loc[('2022-06-01'<=train.date_time)&(train.date_time<'2022-06-02'), 'holiday'] = 1
train.loc[('2022-06-06'<=train.date_time)&(train.date_time<'2022-06-07'), 'holiday'] = 1
train.loc[('2022-08-15'<=train.date_time)&(train.date_time<'2022-08-16'), 'holiday'] = 1
train.head()

,num,date_time,temp,prec,wind,hum,power,hour,day,month,week,day_hour_mean,hour_mean,hour_std,holiday
0,1,20220601 00,18.6,0.0,0.9,42.0,1085.28,0,2,6,22,1774.744615,1706.318118,446.882767,0
1,1,20220601 01,18.0,0.0,1.1,45.0,1047.36,1,2,6,22,1687.347692,1622.620235,439.662704,0
2,1,20220601 02,17.7,0.0,1.5,45.0,974.88,2,2,6,22,1571.483077,1506.971294,412.071906,0
3,1,20220601 03,16.7,0.0,1.4,48.0,953.76,3,2,6,22,1522.153846,1437.365647,391.205981,0
4,1,20220601 04,18.4,0.0,2.8,43.0,986.40,4,2,6,22,1506.793846,1447.321412,381.099697,0


In [19]:
## 시간에 대한 이산푸리에변환 (DFT)
train['sin_time'] = np.sin(2*np.pi*train.hour/24)
train['cos_time'] = np.cos(2*np.pi*train.hour/24)

In [20]:
## 불쾌지수 변수 THI 추가
train['THI'] = 9/5*train['temp'] - 0.55*(1-train['hum']/100)*(9/5*train['hum']-26)+32

기온(temperature) 변수는 설명력이 다소 부족합니다. 냉방 전력소요의 측면에서 보았을 때, 전력 사용량은 기온이 변함에 따라 즉각적으로 변화하진 않기 때문입니다.
냉방전력소요와 관련하여, CDH(cooling degree hour, 냉방도일) 이라는 개념을 조금 변형하여 적용해 보았습니다.  
냉방 기준온도(26도)보다 높은 시점에서 해당 시점의 온도와 기준온도의 차를 적산하여 구할 수 있습니다.  
이렇게 구해진 CDH 라는 변수는, 기온보다 냉방 수요에 대한 설명력이 높습니다. 결과를 시각화 해 보겠습니다.

In [21]:
def CDH(xs):
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)

cdhs = np.array([])
for num in range(1,101,1):
    temp = train[train['num'] == num]
    cdh = CDH(temp['temp'].values)
    cdhs = np.concatenate([cdhs, cdh])
train['CDH'] = cdhs

In [24]:
### 불필요 변수 제거
train.drop(['hour'], axis = 1, inplace = True)
train.head()

,num,date_time,temp,prec,wind,hum,power,day,month,week,day_hour_mean,hour_mean,hour_std,holiday,sin_time,cos_time,THI,CDH
0,1,20220601 00,18.6,0.0,0.9,42.0,1085.28,2,6,22,1774.744615,1706.318118,446.882767,0,0.000000,1.000000,49.6576,-7.4
1,1,20220601 01,18.0,0.0,1.1,45.0,1047.36,2,6,22,1687.347692,1622.620235,439.662704,0,0.258819,0.965926,47.7625,-15.4
2,1,20220601 02,17.7,0.0,1.5,45.0,974.88,2,6,22,1571.483077,1506.971294,412.071906,0,0.500000,0.866025,47.2225,-23.7
3,1,20220601 03,16.7,0.0,1.4,48.0,953.76,2,6,22,1522.153846,1437.365647,391.205981,0,0.707107,0.707107,44.7856,-33.0
4,1,20220601 04,18.4,0.0,2.8,43.0,986.40,2,6,22,1506.793846,1447.321412,381.099697,0,0.866025,0.500000,49.0061,-40.6


In [27]:
## save the preprocessed data
train.to_csv('./data/processing/train_preprocessed.csv')